In [1]:
import pandas as pd

In [2]:
earthquake_counties = pd.read_csv('scrape_earthquake_counties.csv') 
earthquake_counties.head()

,place,county,time,magnitude,latitude,longitude,depth
0,"18km SSE of Lone Pine, CA",Inyo,1593020449240,5.80,36.446833,-117.975167,4.66
1,"17km S of Searles Valley, CA",San Bernardino,1591234331140,5.51,35.614833,-117.428167,8.44
2,"249 km W of Ferndale, California",Humboldt,1589785354591,5.30,40.848300,-127.191800,10.00
3,"251 km W of Ferndale, California",Humboldt,1589784961579,5.00,40.771400,-127.227100,10.00
4,"30km SE of Bodie, CA",Mono,1586615797040,5.24,38.052500,-118.733333,8.45


In [3]:
earthquake_danger = pd.read_csv('earthquake_danger_csv') 
earthquake_danger.head()

,place,county,earthquake_danger_score
0,"18km SSE of Lone Pine, CA",TBD,52.0
1,"17km S of Searles Valley, CA",TBD,52.0
2,"249 km W of Ferndale, California",TBD,52.0
3,"251 km W of Ferndale, California",TBD,52.0
4,"30km SE of Bodie, CA",TBD,52.0


In [4]:
mix = earthquake_counties.join(earthquake_danger, rsuffix="_r")
mix.sort_values(earthquake_danger_score).head()

NameError: name 'earthquake_danger_score' is not defined

In [ ]:
earthquake_danger_score = mix[['place', 'county', 'earthquake_danger_score']]
earthquake_danger_score.head()

In [ ]:
eq_counts = earthquake_danger_score.groupby('county').count()
eq_counts = eq_counts.rename(columns={"place":"earthquakes_per_county"}).drop(columns={'earthquake_danger_score'})
eq_counts

In [ ]:
eq_sums = earthquake_danger_score.groupby('county').sum()
eq_sums

In [ ]:
fire_counties = pd.read_csv('fire_danger_csv') 
fire_counties = fire_counties.round(0)
fire_counties

In [ ]:
fire_counts = fire_counties.groupby('county').count()
fire_counts = fire_counts.rename(columns={"incident_name":"fires_per_county"}).drop(columns={'fire_danger_score'})
fire_counts.head()

In [ ]:
fire_sums = fire_counties.groupby('county').sum()
fire_sums.head()

In [ ]:
disaster_counts = fire_counts.join(eq_counts, how="outer", on="county")
disaster_counts.head()

In [ ]:
disaster_sums = fire_sums.join(eq_sums, how="outer", on="county")
disaster_sums.head()

In [ ]:
all_scores = disaster_counts.join(disaster_sums, how="outer", on="county").fillna(1).reset_index()
all_scores.head()

In [ ]:
i=0
for index, row in all_scores.iterrows():
    fire_freq = round(row[1]/7,2)
    eq_freq = round(row[2]/7,2)
    fire_danger= round(row[3]/7,2)
    eq_danger = round(row[4]/7,2)
    all_scores.at[i,'fire_freq'] = fire_freq
    all_scores.at[i,'fire_danger'] = fire_danger
    all_scores.at[i,'eq_freq'] = eq_freq
    all_scores.at[i,'eq_danger'] = eq_danger
    i=i+1
all_scores.head()

In [ ]:
i=0
for index, row in all_scores.iterrows():
    fire_danger = row[5]*row[6]
    all_scores.at[i,'fire_danger'] = round(fire_danger)
    eq_danger = row[5]*row[6]
    all_scores.at[i,'earthquake_danger'] = round(eq_danger)
    total_danger = (fire_danger+eq_danger)/2
    all_scores.at[i,'total_danger'] = total_danger
    i=i+1
all_scores.sort_values('total_danger', ascending=False).head()

In [ ]:
final_scores = all_scores[['county', 'total_danger']]

final_scores.to_csv('county_danger_scores.csv', index=False)
final = pd.read_csv('county_danger_scores.csv') 
final.head()